# Ice Lakes Detection

Note:
- Use 'Run All' button of notebook to get final testing gpkg in 'GPKG' folder, or just run cells one by one.
- If you need to adjust the batch size to match your GPU memory, change BATCH_SIZE below.
- Training the unet costs for more than one day, and here we provide the model, if you need to train the unet by yourself, change IS_TRAIN. And this will change the old model and lake_polygons_testing.gpkg.

In [1]:
BATCH_SIZE = 64
IS_TRAIN = 0    #0 for just test and 1 for train with test

In [2]:
import geopandas as gpd
import fiona
from osgeo import ogr

import os
import shutil
import numpy as np
import pandas as pd
import wget
import zipfile

import unet
import icelake

np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

## Pre Process


### lake_polygons_training data division

In [3]:
train_gpkg_path = "data/lake_polygons_training.gpkg"
icelake.spilt_training_gpkg(train_gpkg_path)

training_gpkg layers: ['lakes']

splitting training_gpkg by image and region_num:
[2 4 6 1 3 5]
['Greenland26X_22W_Sentinel2_2019-06-03_05.tif'
 'Greenland26X_22W_Sentinel2_2019-06-19_20.tif'
 'Greenland26X_22W_Sentinel2_2019-07-31_25.tif'
 'Greenland26X_22W_Sentinel2_2019-08-25_29.tif']
Saved train_06_03_2
Saved train_07_31_2
Saved train_06_03_4
Saved train_07_31_4
Saved train_06_03_6
Saved train_07_31_6
Saved train_06_19_1
Saved train_08_25_1
Saved train_06_19_3
Saved train_08_25_3
Saved train_06_19_5
Saved train_08_25_5
done
training_gpkg layers after split: ['lakes', 'train_06_03_2', 'train_07_31_2', 'train_06_03_4', 'train_07_31_4', 'train_06_03_6', 'train_07_31_6', 'train_06_19_1', 'train_08_25_1', 'train_06_19_3', 'train_08_25_3', 'train_06_19_5', 'train_08_25_5']


### dem data download

In [4]:
dem_index_zip_path = "data/ArcticDEM_Mosaic_Index_v4_1_gpkg.zip"
if os.path.exists(dem_index_zip_path):
    print('dem index zip already exists')
else:
    wget.download("https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/indexes/ArcticDEM_Mosaic_Index_v4_1_gpkg.zip", dem_index_zip_path)

if not os.path.exists("data/ArcticDEM_Mosaic_Index_v4_1_gpkg.gpkg"):
    with zipfile.ZipFile(dem_index_zip_path, 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print('dem index gpkg already exists')
os.remove(dem_index_zip_path)

dem_folder = "data/dem raster/"
if not os.path.exists(dem_folder):
    os.mkdir(dem_folder)
else:
    print('dem raster folder already exists')

In [5]:
gdf_regions = gpd.read_file("data/lakes_regions.gpkg", layer = "regions")
gdf_dem_index = gpd.read_file("data/ArcticDEM_Mosaic_Index_v4_1_gpkg.gpkg", layer = "ArcticDEM_Mosaic_Index_v4_1_10m")
gdf_regions = gdf_regions.to_crs('EPSG:3413')
intersect = gpd.sjoin(gdf_dem_index, gdf_regions, how="inner", predicate="intersects")
intersect = intersect.drop(columns=["creationdate", "index_right", "region_num"])
intersect = intersect.drop_duplicates()
print(len(intersect), "intersections")

url_1 = "https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/arcticdem/mosaics/v4.1/10m/"

for index, row in intersect.iterrows():
    url = url_1 + row['tile'] + "/" + row['tile'] + "_10m_v4.1_dem.tif"
    path = dem_folder + row['tile'] + "_dem.tif"
    if not os.path.exists(path):
        wget.download(url, path)
        print("downloading dem raster", row['tile'])
    else:
        print(row['tile'], 'dem raster already exists')

intersect = intersect.to_crs('EPSG:3857')
intersect.to_file("data/lakes_regions.gpkg", layer="dem_selected", driver="GPKG", append=True)

32 intersections
downloading dem raster 15_40
downloading dem raster 16_40
downloading dem raster 14_39
downloading dem raster 15_39
downloading dem raster 16_39
downloading dem raster 14_40
downloading dem raster 15_38
downloading dem raster 14_38
downloading dem raster 16_38
downloading dem raster 17_40
downloading dem raster 17_38
downloading dem raster 17_39
downloading dem raster 18_39
downloading dem raster 18_40
downloading dem raster 13_39
downloading dem raster 13_40
downloading dem raster 13_38
downloading dem raster 29_45
downloading dem raster 29_46
downloading dem raster 27_44
downloading dem raster 29_44
downloading dem raster 28_43
downloading dem raster 27_45
downloading dem raster 28_45
downloading dem raster 29_43
downloading dem raster 28_46
downloading dem raster 28_44
downloading dem raster 30_45
downloading dem raster 31_44
downloading dem raster 30_43
downloading dem raster 31_45
downloading dem raster 30_44


### source raster division

In [6]:
folder_path = "data"
source_raster_folder = 'data/source raster'
contents = os.listdir(folder_path)
if not os.path.exists(source_raster_folder):
    os.mkdir(source_raster_folder)
else:
    print('source raster folder already exists')

for item in contents:
    item_path = os.path.join(folder_path, item)
    if 'Greenland' in item:
        try:
            shutil.move(os.path.join(folder_path, item), os.path.join(source_raster_folder))
            print(f"file '{item}' move successfully")
        except Exception as e:
            print(f"error: {str(e)}")


file 'Greenland26X_22W_Sentinel2_2019-06-03_05.tif' move successfully
file 'Greenland26X_22W_Sentinel2_2019-06-03_05.tif.aux.xml' move successfully
file 'Greenland26X_22W_Sentinel2_2019-06-19_20.tif' move successfully
file 'Greenland26X_22W_Sentinel2_2019-06-19_20.tif.aux.xml' move successfully
file 'Greenland26X_22W_Sentinel2_2019-07-31_25.tif' move successfully
file 'Greenland26X_22W_Sentinel2_2019-07-31_25.tif.aux.xml' move successfully
file 'Greenland26X_22W_Sentinel2_2019-08-25_29.tif' move successfully
file 'Greenland26X_22W_Sentinel2_2019-08-25_29.tif.aux.xml' move successfully


In [7]:
regions_gpkg_path = "data/lakes_regions.gpkg"
origan_raster_path = [
    "data/source raster/Greenland26X_22W_Sentinel2_2019-06-03_05.tif",
    "data/source raster/Greenland26X_22W_Sentinel2_2019-06-19_20.tif",
    "data/source raster/Greenland26X_22W_Sentinel2_2019-07-31_25.tif",
    "data/source raster/Greenland26X_22W_Sentinel2_2019-08-25_29.tif",
]
splitted_raster_folder = "data/splitted raster/"
if not os.path.exists(splitted_raster_folder):
    os.mkdir(splitted_raster_folder)
else:
    print('splitted raster folder already exists')
icelake.split_source_raster(
    regions_gpkg_path, origan_raster_path, splitted_raster_folder
)

06_03_1.tif done.
06_03_2.tif done.
06_03_3.tif done.
06_03_4.tif done.
06_03_5.tif done.
06_03_6.tif done.
06_19_1.tif done.
06_19_2.tif done.
06_19_3.tif done.
06_19_4.tif done.
06_19_5.tif done.
06_19_6.tif done.
07_31_1.tif done.
07_31_2.tif done.
07_31_3.tif done.
07_31_4.tif done.
07_31_5.tif done.
07_31_6.tif done.
08_25_1.tif done.
08_25_2.tif done.
08_25_3.tif done.
08_25_4.tif done.
08_25_5.tif done.
08_25_6.tif done.


### calculate NDWI-Ice

In [8]:
ndwi_ice_folder = "data/ndwi ice/"
if not os.path.exists(ndwi_ice_folder):
    os.mkdir(ndwi_ice_folder)
else:
    print('ndwi ice folder already exists')

for dirname, _, filenames in os.walk("data/splitted raster/"):
    filenames.sort()
    for filename in filenames:
        raster_path = os.path.join(dirname, filename)
        icelake.cal_ndwi(raster_path, ndwi_ice_folder)

06_03_1.tif NDWI_ice calculation and saving completed.
06_03_2.tif NDWI_ice calculation and saving completed.
06_03_3.tif NDWI_ice calculation and saving completed.
06_03_4.tif NDWI_ice calculation and saving completed.
06_03_5.tif NDWI_ice calculation and saving completed.
06_03_6.tif NDWI_ice calculation and saving completed.
06_19_1.tif NDWI_ice calculation and saving completed.
06_19_2.tif NDWI_ice calculation and saving completed.
06_19_3.tif NDWI_ice calculation and saving completed.
06_19_4.tif NDWI_ice calculation and saving completed.
06_19_5.tif NDWI_ice calculation and saving completed.
06_19_6.tif NDWI_ice calculation and saving completed.
07_31_1.tif NDWI_ice calculation and saving completed.
07_31_2.tif NDWI_ice calculation and saving completed.
07_31_3.tif NDWI_ice calculation and saving completed.
07_31_4.tif NDWI_ice calculation and saving completed.
07_31_5.tif NDWI_ice calculation and saving completed.
07_31_6.tif NDWI_ice calculation and saving completed.
08_25_1.ti

## U-Net

### training

In [9]:
layers = fiona.listlayers("data/lake_polygons_training.gpkg")
if IS_TRAIN == 1:
    train_tif, train_mask, val_tif, val_mask, lakes_true, tif_array = icelake.data_preprocess_train([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [], layers[1:], train_with_all_data=False)
    print(f"{train_tif.shape}\t{train_mask.shape}")
    sliced_image_set = unet.run_unet(train_tif, train_mask, val_tif, val_mask, print_tqdm=False, BATCH_SIZE=BATCH_SIZE)

In [10]:
# # val model, no need in testing
# layers = fiona.listlayers("data/lake_polygons_training.gpkg")
# gdf_pred_all = gpd.GeoDataFrame()
# kf = KFold(n_splits=len(layers[1:]), shuffle=True, random_state=42)
# for train_layer_indices, val_layer_indices in kf.split(layers[1:]):
#     tif_num = re.search(r"train_(.+)", layers[val_layer_indices[0] + 1]).group(1)

#     train_tif, train_mask, val_tif, val_mask, lakes_true, tif_array = icelake.data_preprocess_train([], val_layer_indices, layers[1:], train_with_all_data=False)
#     print(f"Val {tif_num}: {val_tif.shape}\t{val_mask.shape}\t{lakes_true.shape}")
#     sliced_image_set = unet.run_unet(train_tif, train_mask, val_tif, val_mask, print_tqdm=False, BATCH_SIZE=BATCH_SIZE)
#     lakes_pred = icelake.splice_image(lakes_true, sliced_image_set)
#     del train_tif, train_mask, val_tif, val_mask, sliced_image_set

#     # lakes_pred post process in tif
#     tmp_mask = np.ones_like(lakes_pred)
#     tmp_mask[np.all(tif_array == 0, axis=0)] = 0
#     lakes_pred = lakes_pred & tmp_mask
#     # lakes_pred post process in gpkg

#     gdf_pred = icelake.array2gdf(lakes_pred, tif_num)
#     del lakes_true, lakes_pred, tif_array, tmp_mask

#     gdf_true = gpd.read_file(
#         "data/lake_polygons_training.gpkg", layer="train_" + tif_num
#     )
#     icelake.cal_scores_vector(gdf_true, gdf_pred)

#     gdf_pred = icelake.post_process(gdf_pred, tif_num)
#     gdf_pred["tif_num"] = tif_num
#     gdf_pred_all = pd.concat([gdf_pred_all, gdf_pred], ignore_index=True)
    
#     icelake.cal_scores_vector(gdf_true, gdf_pred)

### testing with post process

In [11]:
if not os.path.exists("GPKG/lake_polygons_testing.gpkg"):
    gpkg_ds = ogr.GetDriverByName('GPKG').CreateDataSource("GPKG/lake_polygons_testing.gpkg")
    gpkg_ds = None

In [12]:
layers = [
    "06_03_1",
    "06_03_3",
    "06_03_5",
    "06_19_2",
    "06_19_4",
    "06_19_6",
    "07_31_1",
    "07_31_3",
    "07_31_5",
    "08_25_2",
    "08_25_4",
    "08_25_6",
]
gdf_pred_all = gpd.GeoDataFrame()
for tif_num in layers:
    test_tif, test_mask, tif_array, ndwi_array = icelake.data_preprocess_test(tif_num)
    print(f"Test {tif_num}: {test_tif.shape}\t{test_mask.shape}\t{tif_array.shape}")
    sliced_image_set = unet.run_unet([], [], test_tif, test_mask, print_tqdm=False, BATCH_SIZE=BATCH_SIZE)
    lakes_pred = icelake.splice_image(ndwi_array, sliced_image_set)
    del test_tif, test_mask, sliced_image_set

    # lakes_pred post process in tif
    tmp_mask = np.ones_like(lakes_pred)
    tmp_mask[np.all(tif_array == 0, axis=0)] = 0
    lakes_pred = lakes_pred & tmp_mask
    # lakes_pred post process in gpkg

    gdf_pred = icelake.array2gdf(lakes_pred, tif_num)
    del lakes_pred, tif_array, tmp_mask

    gdf_pred = icelake.post_process(gdf_pred, tif_num)
    gdf_pred["tif_num"] = tif_num
    gdf_pred_all = pd.concat([gdf_pred_all, gdf_pred], ignore_index=True)
    

Test 06_03_1: (6936, 256, 256, 3)	(6936, 256, 256)	(3, 8731, 13076)
=> Loading Checkpoint
Test 06_03_3: (8400, 256, 256, 3)	(8400, 256, 256)	(3, 9721, 14412)
=> Loading Checkpoint
Test 06_03_5: (45684, 256, 256, 3)	(45684, 256, 256)	(3, 24192, 31214)
=> Loading Checkpoint
Test 06_19_2: (7888, 256, 256, 3)	(7888, 256, 256)	(3, 8708, 14955)
=> Loading Checkpoint
Test 06_19_4: (8400, 256, 256, 3)	(8400, 256, 256)	(3, 10356, 13522)
=> Loading Checkpoint
Test 06_19_6: (38720, 256, 256, 3)	(38720, 256, 256)	(3, 22639, 28233)
=> Loading Checkpoint
Test 07_31_1: (6936, 256, 256, 3)	(6936, 256, 256)	(3, 8731, 13076)
=> Loading Checkpoint
Test 07_31_3: (8400, 256, 256, 3)	(8400, 256, 256)	(3, 9721, 14412)
=> Loading Checkpoint
Test 07_31_5: (45684, 256, 256, 3)	(45684, 256, 256)	(3, 24192, 31214)
=> Loading Checkpoint
Test 08_25_2: (7888, 256, 256, 3)	(7888, 256, 256)	(3, 8708, 14955)
=> Loading Checkpoint
Test 08_25_4: (8400, 256, 256, 3)	(8400, 256, 256)	(3, 10356, 13522)
=> Loading Checkpoint

### save testing result

In [13]:
def get_image_name(tif_num):
    tif_num = tif_num[:5]
    if tif_num == '06_03':
        return "Greenland26X_22W_Sentinel2_2019-06-03_05.tif"
    elif tif_num == '06_19':
        return "Greenland26X_22W_Sentinel2_2019-06-19_20.tif"
    elif tif_num == '07_31':
        return "Greenland26X_22W_Sentinel2_2019-07-31_25.tif"
    elif tif_num == '08_25':
        return "Greenland26X_22W_Sentinel2_2019-08-25_29.tif"
    else:
        raise ValueError
    
gdf_pred_all["region_num"] = (gdf_pred_all["tif_num"]).apply(lambda x: x[-1])
gdf_pred_all["image"] = gdf_pred_all["tif_num"].apply(lambda x: get_image_name(x))
gdf_pred_all = gdf_pred_all.drop(columns=["tif_num"])

gdf_pred_all.to_file("GPKG/lake_polygons_testing.gpkg", layer="lakes", driver='GPKG')
print(f"{len(gdf_pred_all)} lakes saved to path GPKG/lake_polygons_testing.gpkg")

5316 lakes saved to path GPKG/lake_polygons_testing.gpkg
